### Loading Required Libraries

In [2]:
# import the necessary libraries

from modules import PreProcessing, functions
from sqlite3 import connect 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from itertools import chain
from collections import Counter
import nltk
import warnings
warnings.filterwarnings("ignore")

# to ensure all changes are reflected 
# in-case jupyter notebook was already oppened

from importlib import reload
reload(PreProcessing)
reload(functions)

<module 'modules.functions' from '/Volumes/GoogleDrive/My Drive/_509/projekt_509/modules/functions.py'>

In [3]:
import spacy.cli
spacy.cli.download("en_core_web_md")

import spacy_fastlang

import spacy
from bertopic import BERTopic


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


### Section 1 - Create a Suport vector machine 

In [6]:
databse_file = 'database.db'
main = connect(databse_file)

sql_statement_laod_data = '''
SELECT 
    reviews.id, 
    reviews.course_name,
    reviews.text

FROM reviews
'''

df = pd.read_sql_query(sql_statement_laod_data, main)

df.sample(5)

,id,course_name,text
33898,33898,python-network-data,Excellent course to get started in handling da...
56134,56134,learning-how-to-learn,I really enjoyed this course. The lecturers ca...
29849,29849,introduction-cybersecurity-cyber-attacks,A very detailed intro course to cybersecurity ...
28647,28647,foundations-data,Really enjoyed
30037,30037,introduction-cybersecurity-cyber-attacks,A very nice Course to take if really want to u...


In [21]:
df.isnull().sum() * 100 / len(df)

id             0.000000
course_name    0.000000
text           0.049845
dtype: float64

In [22]:
df.dropna(inplace=True)

In [23]:
df.isnull().sum() * 100 / len(df)

id             0.0
course_name    0.0
text           0.0
dtype: float64

In [24]:
# dfX = df.copy()

In [25]:
# OOOPSIES
# df = dfX.copy()

In [26]:
import re
def tokens_2_keep(text_column):
    tokens = text_column.split()
    return [token for token in tokens if re.match('^[a-z0-9]+$', token)]

def lang_detect(clean_string):
    clean_string = nlp(clean_string)
    return clean_string._.language

nlp = spacy.load('en_core_web_md', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])
nlp.add_pipe("language_detector")

In [27]:
%%time
df['language'] = df['text'].apply(lang_detect)
df = df[df['language'] == 'en']
df.sample(5)

CPU times: user 2min 27s, sys: 3.08 s, total: 2min 30s
Wall time: 2min 39s


,id,course_name,text,language
54218,54218,learning-how-to-learn,I highly recommend this to anyone who has trou...,en


In [29]:
def token_counter(text):
    return len(text.split())


In [36]:
# df['token_count'] = df['text'].apply(token_counter)
# check to see if threshold is fine
df[df['token_count'] == 1].sample(10)

,id,course_name,text,language,token_count
32103,32103,introduction-cybersecurity-cyber-attacks,Excellent.,en,1
7946,7946,ai-for-everyone,Great,en,1
39877,39877,python-network-data,great,en,1
8039,8039,ai-for-everyone,Good,en,1
8196,8196,ai-for-everyone,good,en,1
39906,39906,python-network-data,Good,en,1
38359,38359,python-network-data,Superb!!!\n,en,1
7625,7625,ai-for-everyone,Excellent!,en,1
7853,7853,ai-for-everyone,Awesome,en,1
40206,40206,python-network-data,thanks,en,1


In [32]:
df['english_tokens'] = df['tokens_keep'].apply(tokens_2_keep)
df['token_count'] = df['english_tokens'].apply(len)

df = df[['id', 'clean_sentence', 'course_name', 'english_tokens','token_count']]
df.head(2)

,id,clean_sentence,course_name,english_tokens,token_count
50472,50472,helloâlearn learnâ first might seem quit a...,learning-how-to-learn,"[first, might, seem, quit, assur, thi, would, ...",173
50488,50488,wa permit would rate 10 star thi cours hail in...,learning-how-to-learn,"[wa, permit, would, rate, 10, star, thi, cours...",121


In [28]:
# check to see if threshold is fine
df[df['token_count'] > 5].loc[df['token_count'] < 8].sample(10)

KeyError: 'token_count'

It appears we should keep anything above 5 tokens.

In [35]:
df = df[df['token_count'] >5]
df.shape

(39750, 5)

In [41]:
' '.join(df.loc[0, 'english_tokens'])

'got comprehens overview ai mean variou concept talk thi context excel cours one start solid ground five star thank andrew'

In [ ]:
df['tokens_sentence']

# Model

In [36]:
nlp = spacy.load('en_core_web_md', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

In [37]:
topic_model = BERTopic(embedding_model=nlp, nr_topics='auto')
topics, probs = topic_model.fit_transform(df['clean_sentence'])


KeyboardInterrupt: 

In [ ]:
fig = topic_model.visualize_topics()
fig.show()